In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

In [ ]:
def cal_loss(img_crp1,img_crp2,blk_sz):
  num_blk_y=int(img_crp1.shape[0]/blk_sz)
  num_blk_x=int(img_crp1.shape[1]/blk_sz)
  dif=0
  for nth_y in range(num_blk_y):
    for nth_x in range(num_blk_x):
      i1c1=np.mean(img_crp1[nth_y*blk_sz:(nth_y+1)*blk_sz-1,nth_x*blk_sz:(nth_x+1)*blk_sz-1:,0])
      i1c2=np.mean(img_crp1[nth_y*blk_sz:(nth_y+1)*blk_sz-1,nth_x*blk_sz:(nth_x+1)*blk_sz-1:,1])
      i1c3=np.mean(img_crp1[nth_y*blk_sz:(nth_y+1)*blk_sz-1,nth_x*blk_sz:(nth_x+1)*blk_sz-1:,2])
      i2c1=np.mean(img_crp2[nth_y*blk_sz:(nth_y+1)*blk_sz-1,nth_x*blk_sz:(nth_x+1)*blk_sz-1:,0])
      i2c2=np.mean(img_crp2[nth_y*blk_sz:(nth_y+1)*blk_sz-1,nth_x*blk_sz:(nth_x+1)*blk_sz-1:,1])
      i2c3=np.mean(img_crp2[nth_y*blk_sz:(nth_y+1)*blk_sz-1,nth_x*blk_sz:(nth_x+1)*blk_sz-1:,2])
      dif=dif+abs(i1c1-i2c1)+abs(i1c2-i2c2)+abs(i1c3-i2c3)
  return dif
def loss4img(img1,img2,x,y,blk_sz):
  if abs(y)>max(img1.shape[0],img2.shape[0]) or abs(x)>max(img1.shape[1],img2.shape[1]):
    return False
  Area=(img1.shape[0]-abs(x))*(img1.shape[1]-abs(y))
  img_crp1=img1[max(0,y):min(img1.shape[0],img1.shape[0]+y),max(0,x):min(img1.shape[1],img1.shape[1]+x),:]
  img_crp2=img2[max(0,-y):min(img2.shape[0]-y,img2.shape[0]),max(0,-x):min(img2.shape[1]-x,img2.shape[1]),:]
  result=cal_loss(img_crp1,img_crp2,blk_sz)/Area
  return result

In [ ]:
def ntrl_sel(img1,img2,parents_xy,x_min,x_max,y_min,y_max,step,num_children):
  candidates=[]
  score=[]
  children=[]
  num_x_blk=int((x_max-x_min)/step)
  num_y_blk=int((y_max-y_min)/step)
  for point_id in range(len(parents_xy)):
    point=parents_xy[point_id]
    for x_blk in range(num_x_blk+1):
      for y_blk in range(num_y_blk+1):
        candidates.append((point[0]+x_min+x_blk*step,point[1]+y_min+y_blk*step))
  for num_cand in range(len(candidates)):
    cur_score=loss4img(img1,img2,candidates[num_cand][0],candidates[num_cand][1],10)
    score.append(cur_score)
  score_ar=np.array(score)
  index=np.argpartition(score_ar,num_children)
  selected_idx=index[:num_children]
  for selected_cand in range(len(selected_idx)):
    children.append(candidates[selected_idx[selected_cand]])
  return children


In [ ]:
def genetic_img(img1,img2):
  root_xy=[(0,0)]
  gen1=ntrl_sel(img1,img2,root_xy,-100,300,-100,100,50,3)
  print(gen1)
  gen2=ntrl_sel(img1,img2,gen1,-20,20,-20,20,10,6)
  print(gen2)
  gen3=ntrl_sel(img1,img2,gen2,-4,4,-4,4,2,10)
  print(gen3)
  x_sum=0
  y_sum=0
  for pt in range(len(gen3)):
    x_sum=x_sum+gen3[pt][0]
    y_sum=y_sum+gen3[pt][1]
  off_x=int(x_sum/len(gen3))
  off_y=int(y_sum/len(gen3))
  return off_x,off_y

In [ ]:
img1=cv2.imread('/content/drive/MyDrive/Colab Notebooks/Panorama/data/012L.png')
img2=cv2.imread('/content/drive/MyDrive/Colab Notebooks/Panorama/data/012R.png')
result=genetic_img(img1,img2)
print(result)

[(200, 0), (200, -50), (150, 0)]
[(190, -10), (200, -10), (200, -20), (190, -20), (200, 0), (190, 0)]
[(196, -10), (196, -12), (198, -10), (196, -8), (194, -12), (194, -14), (196, -14), (194, -10), (198, -12), (194, -8)]
(195, -11)


In [ ]:
img1=cv2.imread('/content/drive/MyDrive/Colab Notebooks/Panorama/data/001L.png')
img2=cv2.imread('/content/drive/MyDrive/Colab Notebooks/Panorama/data/001R.png')
result=genetic_img(img1,img2)
print(result)

[(0, -100), (150, -100), (150, -50)]
[(140, -70), (150, -80), (150, -70), (140, -80), (140, -90), (130, -80)]
[(142, -82), (144, -82), (146, -78), (146, -80), (144, -80), (144, -78), (144, -76), (142, -78), (142, -80), (142, -76)]
(143, -79)


In [ ]:
img1=cv2.imread('/content/drive/MyDrive/Colab Notebooks/Panorama/data/040L.png')
img2=cv2.imread('/content/drive/MyDrive/Colab Notebooks/Panorama/data/040R.png')
result=genetic_img(img1,img2)
print(result)

[(200, 0), (200, -50), (150, 0)]
[(210, -10), (200, 0), (200, -10), (210, 0), (200, -20), (190, -10)]
[(204, -4), (204, -8), (204, -6), (202, -6), (206, -6), (206, -4), (202, -8), (202, -4), (206, -8), (204, -2)]
(204, -5)


In [ ]:
img1=cv2.imread('/content/drive/MyDrive/Colab Notebooks/Panorama/data/053L.png')
img2=cv2.imread('/content/drive/MyDrive/Colab Notebooks/Panorama/data/053R.png')
result=genetic_img(img1,img2)
print(result)

[(150, 0), (150, 50), (200, 0)]
[(150, 0), (150, 10), (160, 10), (150, 20), (140, 10), (160, 0)]
[(152, 8), (152, 12), (154, 8), (150, 8), (150, 10), (154, 10), (152, 10), (152, 6), (150, 12), (150, 6)]
(151, 9)


In [ ]:
img1=cv2.imread('/content/drive/MyDrive/Colab Notebooks/Panorama/data/135L.png')
img2=cv2.imread('/content/drive/MyDrive/Colab Notebooks/Panorama/data/135R.png')
result=genetic_img(img1,img2)
print(result)

[(200, 0), (200, -50), (150, 0)]
[(190, -20), (180, -20), (200, -20), (200, -30), (190, -30), (210, -20)]
[(196, -24), (192, -24), (196, -22), (194, -22), (194, -24), (192, -22), (194, -20), (196, -20), (194, -26), (198, -22)]
(194, -22)


In [ ]:
img1=cv2.imread('/content/drive/MyDrive/Colab Notebooks/Panorama/data/169L.png')
img2=cv2.imread('/content/drive/MyDrive/Colab Notebooks/Panorama/data/169R.png')
result=genetic_img(img1,img2)
print(result)

[(200, 0), (250, 0), (150, 0)]
[(230, 10), (220, 10), (230, 20), (240, 10), (230, 0), (220, 20)]
[(228, 14), (230, 12), (226, 10), (226, 12), (226, 14), (230, 10), (228, 8), (228, 10), (228, 12), (226, 8)]
(227, 11)


In [ ]:
img1=cv2.imread('/content/drive/MyDrive/Colab Notebooks/Panorama/data/183L.png')
img2=cv2.imread('/content/drive/MyDrive/Colab Notebooks/Panorama/data/183R.png')
result=genetic_img(img1,img2)
print(result)

[(150, 0), (100, 0), (200, 0)]
[(180, -10), (170, -10), (170, 0), (190, -10), (160, -10), (160, 0)]
[(178, -10), (174, -10), (176, -10), (176, -8), (174, -8), (178, -8), (172, -8), (180, -10), (180, -8), (172, -10)]
(176, -9)


In [ ]:
img1=cv2.imread('/content/drive/MyDrive/Colab Notebooks/Panorama/data/202L.png')
img2=cv2.imread('/content/drive/MyDrive/Colab Notebooks/Panorama/data/202R.png')
result=genetic_img(img1,img2)
print(result)

[(200, 0), (150, 0), (100, 0)]
[(220, 0), (210, 0), (200, 0), (190, 0), (200, 10), (200, -10)]
[(202, 0), (200, -2), (202, -4), (204, 0), (204, -2), (202, -2), (200, 0), (202, 2), (206, -2), (200, 2)]
(202, 0)
